In [1]:
import theano
import numpy as np

import logging
log = logging.getLogger()
log.setLevel("DEBUG")
from braindecode.scripts.train_experiments import setup_logging
setup_logging()

from braindecode.veganlasagne.layer_util import print_layers
import lasagne

import os

%load_ext autoreload
%autoreload 2

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)
/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import receptive_field

In [3]:
modelpath = '/home/hartmank/braindecode/data/models/'
modelname = 'paper/ours/cnt/deep4/car/22'
savepath  = '/home/hartmank/data/convvisual/RF_data'

In [4]:
#%cd /home/hartmank/braindecode
RF_extractor = receptive_field.ReceptiveFieldExtractor(os.path.join(modelpath,modelname))
#%cd /home/hartmank/braindecode/convvisual

Init extractor
2017-01-21 19:08:39,772 Setting n_sample preds automatically to 479
2017-01-21 19:08:39,773 Input window length is 522
2017-01-21 19:08:40,016 Setting n_sample preds automatically to 479
2017-01-21 19:08:40,017 Input window length is 522
2017-01-21 19:08:40,018 Load Training Set...
2017-01-21 19:09:11,357 Load Test Set...
2017-01-21 19:09:17,693 Clean Training Set...
2017-01-21 19:09:18,919 Rejected channels: []
2017-01-21 19:09:18,920 #Clean trials:     895
2017-01-21 19:09:18,921 #Rejected trials:  2
2017-01-21 19:09:18,921 Fraction Clean:    99.0%
2017-01-21 19:09:18,922 (from maxmin):     2
2017-01-21 19:09:18,923 (from var):        0
2017-01-21 19:09:19,253 Clean Test Set...
2017-01-21 19:09:19,469 Rejected channels: []
2017-01-21 19:09:19,470 #Clean trials:     160
2017-01-21 19:09:19,470 #Rejected trials:  0
2017-01-21 19:09:19,471 Fraction Clean:    100.0%
2017-01-21 19:09:19,471 (from maxmin):     0
2017-01-21 19:09:19,472 (from var):        0
2017-01-21 19:09:1

In [5]:
if not os.path.isdir(os.path.join(savepath,modelname)):
    os.makedirs (os.path.join(savepath,modelname))

In [6]:
from braindecode.veganlasagne.layers import FinalReshapeLayer
model = RF_extractor.model

all_l = lasagne.layers.get_all_layers(model)
model = all_l[-3]
model = FinalReshapeLayer(model,remove_invalids=True,flatten=False)
model = lasagne.layers.DimshuffleLayer(model,(0,2,1,3))
model = lasagne.layers.ReshapeLayer(model,(-1,4))
model = lasagne.layers.NonlinearityLayer(model,nonlinearity=lasagne.nonlinearities.softmax)

RF_extractor.set_model(model)

In [7]:
print_layers(model)

 0-InputLayer                                                         [None, 128, 1000, 1]
 1-DimshuffleLayer                                                    (None, 1, 1000, 128)
 2-DropoutLayer             0.000                                    
 3-Conv2DLayer              10x1                                      (None, 25, 991, 128)
 4-Conv2DAllColsLayer       1x128                                     (None, 25, 991, 1)
 5-BatchNormLayer                                     elu            
 6-Pool2DLayer              3x1                       max             (None, 25, 989, 1)
 7-StrideReshapeLayer           ::3 ::1                               (None, 25, 330, 1)
 8-NonlinearityLayer                                                 
 9-DropoutLayer             0.500                                    
10-Conv2DLayer              10x1                                      (None, 50, 321, 1)
11-BatchNormLayer                                     elu            
12-Pool2DLayer       

In [8]:
#n_Units = 30
#n_filters = None

#layer_list = [3,5,11,17,23,28]
#X_reshapes = [None,None,3,9,27,81]
#splits = [1,1,4,4,8,8]

#RF_save = receptive_field.ReceptiveFieldInputsIO()
#for layer,X_reshape,split in zip(layer_list,X_reshapes,splits):
#    RF_save.results = list()
#    RF_inputs = RF_extractor.calc_max_RF_input(layer,X_reshape,
#                                                  None,n_filters,
#                                                  split,n_Units,False)
#    RF_save.results.append(RF_inputs)
        
#    RF_save.save(os.path.join(savepath,modelname,
#                              'Layer%02d_nUnits%02d_nFiltersNone_notsorted.data'%(layer,n_Units)))

In [9]:
datasets = RF_extractor.exp.dataset_provider.get_train_merged_valid_test(RF_extractor.exp.dataset)
RF_extractor.exp.iterator.batch_size = 999999
test_batches = list(RF_extractor.exp.iterator.get_batches(datasets['train'], shuffle=False))
inputs,targets = test_batches[0]

In [ ]:
print inputs.shape
print RF_extractor.targets.shape

(1790, 128, 1000, 1)
(1790,)


In [ ]:
n_Units = 200
n_filters = 5

layer_list = [3,4,10,16,22,28]#[8,14,20,26]
X_reshapes = [None,None,3,9,27,81]#[3,9,27,81]
splits = [20,20,30,30,40,40]#[1,4,4,8]
classes = [0,1,2,3]

RF_save = receptive_field.ReceptiveFieldInputsIO()
for layer,X_reshape,split in zip(layer_list,X_reshapes,splits):  
    RF_save.classes = classes
    RF_save.results = list()
    for c in classes:
        ind_input = RF_extractor.targets==c
        RF_inputs = RF_extractor.calc_max_RF_input(layer,X_reshape,
                                                  ind_input,n_filters,
                                                  split,n_Units,True)
        RF_save.results.append(RF_inputs)
        
    RF_save.save(os.path.join(savepath,modelname,
                              'Layer%02d_nUnits%02d_nFilters%02d_filterdiff_traindata.data'%(layer,n_Units,n_filters)))

/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/helper.py:212: UserWarning: get_output() was called with unused kwargs:
	input_var
  % "\n\t".join(suggestions))


Output
Output1


In [ ]:
RF_extractor.inputs.shape